In [42]:
import praw
import pandas as pd
import datetime as dt
import numpy as np
import time
%matplotlib inline

In [20]:
reddit = praw.Reddit(client_id='Bb1fhVsMwX10GQ', \
                     client_secret='Tj5dnHfyUjftHdbPtZLP6fbtH_4', \
                     user_agent="Hill-Brilly's", \
                     username='RockDrums11', \
                     password='hillbrillies69')

In [51]:
subreddits = pd.read_csv("subreddits.csv", header = None)

In [52]:
subreddits.shape

(31, 1)

In [53]:
subreddits.columns = ['subreddit']
df = pd.DataFrame(['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'comments', 'created'])

In [54]:
%%time
posts = []
for subreddit in subreddits['subreddit']:
    current_subreddit = reddit.subreddit(subreddit)
    print(subreddit)
    for post in current_subreddit.hot(limit=100):
        posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.comments, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'comments', 'created'])
posts.head()

Capitalism
changemyview
Communist
Conservative
conservatives
democrats
ElizabethWarren
Fuckthealtright
government
JoeBiden
Liberal
Libertarian
LibertarianLeft
Marxism
neoliberal
Pete_Buttigieg
PoliticalDiscussion
PoliticalHumor
politics
progressive
RadicalFeminism
Republican
republicanism
republicans
SandersForPresident
socialdemocracy
socialism
The_Donald
uspolitics
VoteBlue
YangForPresidentHQ
Wall time: 54min 36s


,title,score,id,subreddit,url,num_comments,comments,created
0,Community Comments,22,dztn6a,Capitalism,https://www.reddit.com/r/Capitalism/comments/d...,2,"(f8ah9li, f8ahgtb)",1.574415e+09
1,[rant] Socialist Myths: Each time it ends in p...,34,e01v3l,Capitalism,https://www.reddit.com/r/Capitalism/comments/e...,4,"(f8buptb, f8c60zv, f8c97wh, f8bvm02)",1.574463e+09
2,Costco vs Amazon: How Costco is beating Amazon?,11,e04g19,Capitalism,https://youtu.be/8F0Ynb05UTg,1,(f8bww5n),1.574474e+09
3,"Gary Kasparov Speaks the Truth, Entitled Ameri...",64,dzw56i,Capitalism,https://www.reddit.com/r/Capitalism/comments/d...,39,"(f8aq6h8, f8b64xg, f8b9jy4, f8bqlm4, f8bw3cj, ...",1.574427e+09
4,Milton Friedman Describes Hong Kong as an Exam...,1,e07mh8,Capitalism,https://www.youtube.com/watch?v=xqh0zXSd4vc&fe...,0,(),1.574487e+09


In [56]:
posts.astype({'subreddit': 'str'}).groupby('subreddit')['num_comments'].agg(sum).sort_values(ascending = False).sum()

129913

In [57]:
df_test = posts[:10]

In [84]:
from praw.models import MoreComments

def extract_comments(s):
    ret_list = []
    for comment in s:
        if isinstance(comment, MoreComments):
            continue
        ret_list.append(comment.body)
    return ret_list

In [85]:
test = df_test['comments'].apply(extract_comments)

In [83]:
test = [x for x in test if x != []]
print(type(test))

<class 'list'>


In [66]:
for comment in df_test['comments'][0]:
    if isinstance(comment, MoreComments):
        continue
    print(comment.body)

Deregulation and market selection are the **only** ways to run a sub about capitalism! Let those losers at /r/chapotraphouse “ban” things they don’t like! 

All markets have a risk of failure due to unfair manipulation. We should only moderate to keep the system functioning. That means “no” to moderating content—good and bad discussions and topics can rise of fall on their votes—but “yes” to moderating illegal content, posts that violate Reddit TOS, and spam (eg solicitations). That’s the way to go!

Thanks for your efforts!
Hey, if you need mods i am willing to be one. I have prior moving experience and believe that capitalism is the right way to go as well as I'm all for civil discussion. I might be a good fit.


In [86]:
posts.to_csv("first_scrape.csv")